In [1]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook uses the mpilot library to build and run the EEMS model. The output is then joined to the original dataset using the `join_id` field using the Geopandas libary

**Input Data Layers:**
- Summarized Reporting Units (created in `4_Summarize.ipynb`)
    - Need both the csv and the shapefile


In [2]:
df = pd.read_csv("out_data/eems/A_RU_9.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18180 entries, 0 to 18179
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Join_ID     18180 non-null  int64  
 1   apn         18179 non-null  object 
 2   Pr_Lg_Fr    18180 non-null  float64
 3   Pr_Fire_Ig  18180 non-null  float64
 4   Mj_LU_Cat   18180 non-null  int64  
 5   WUI_Interm  18180 non-null  int64  
 6   WUI_Interf  18180 non-null  int64  
 7   sa_emb_rsk  18180 non-null  float64
 8   sd_emb_rsk  18180 non-null  float64
 9   bl_dns_0p5  18180 non-null  float64
 10  bl_dns_3mi  18180 non-null  float64
 11  Shape_Leng  18180 non-null  float64
 12  Shape_Area  18180 non-null  float64
 13  bl_dns_1kf  18180 non-null  float64
 14  geometry    18180 non-null  object 
 15  LU_Cat      18180 non-null  float64
dtypes: float64(10), int64(4), object(2)
memory usage: 2.2+ MB


**Create the mpilot command file**

In [31]:
def get_base_str(reporting_units_path, output_path):
    base_str = f"""

# Read Commands
Join_ID = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Join_ID",
    DataType = "Integer",
    Metadata = [
        DisplayName: Join&nbsp;ID,
        Description: Join&nbsp;ID,
        ColorMap: RdYlBu_r
    ]
)

Mj_LU_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_LU_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Majority&nbsp;LandUse&nbsp;Category,
        Description: Majority&nbsp;LandUse&nbsp;Category,
        ColorMap: RdYlBu_r
    ]
)

Pr_Lg_Fr = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Lg_Fr",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        Description: Relative&nbsp;Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Fire_Ig = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Fire_Ig",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interm = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interm",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Intermix,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interf",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Interface,
        ColorMap: RdYlBu_r
    ]
)

sa_emb_rsk = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "sa_emb_rsk",
    DataType = "Integer",
    Metadata = [
        DisplayName: Santa&nbsp;Ana&nbsp;Ember&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

sd_emb_rsk = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "sd_emb_rsk",
    DataType = "Integer",
    Metadata = [
        DisplayName: Sundowner&nbsp;Ember&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

bl_dns_0p5 = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "bl_dns_0p5",
    DataType = "Integer",
    Metadata = [
        DisplayName: Half&nbsp;Mile&nbsp;Building&nbsp;Density&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

bl_dns_3mi = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "bl_dns_3mi",
    DataType = "Integer",
    Metadata = [
        DisplayName: Three&nbsp;Mile&nbsp;Building&nbsp;Density&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

bl_dns_1kf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "bl_dns_1kf",
    DataType = "Integer",
    Metadata = [
        DisplayName: Thousand&nbsp;Feet&nbsp;Building&nbsp;Density&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_LU_Cat, 
    RawValues = [0,1,2,3,4], 
    FuzzyValues = [1,0,0.1,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Fuzzy&nbsp;LandUse&nbsp;Category,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Cvt = CvtFromFuzzy(
    InFieldName = LU_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Converted&nbsp;LandUse&nbsp;Category,
        ColorMap: RdYlBu_r
    ]
)



# Normalizations

Nm_Lg_Fire = Normalize(
    InFieldName = Pr_Lg_Fr, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fire_Ig = Normalize(
    InFieldName = Pr_Fire_Ig, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

nm_dns_1kf = Normalize(
    InFieldName = bl_dns_1kf, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Thousand&nbsp;Feet&nbsp;Building&nbsp;Density,
        ColorMap: RdYlBu_r
    ]
)

nm_dns_0p5 = Normalize(
    InFieldName = bl_dns_0p5, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Half&nbsp;Mile&nbsp;Building&nbsp;Density,
        ColorMap: RdYlBu_r
    ]
)

nm_dns_3mi = Normalize(
    InFieldName = bl_dns_3mi, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Three&nbsp;Mile&nbsp;Building&nbsp;Density,
        ColorMap: RdYlBu_r
    ]
)

nm_sa_rsk = Normalize(
    InFieldName = sa_emb_rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Santa&nbsp;Ana&nbsp;Ember&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

nm_sd_rsk = Normalize(
    InFieldName = sd_emb_rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Sundowner&nbsp;Ember&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

# Operations

embr_rsk = WeightedSum(
    InFieldNames = [nm_sd_rsk, nm_sa_rsk], 
    Weights = [0.5, 0.5],
    Metadata = [
        DisplayName: Combined&nbsp;Ember&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

nm_emb_rsk = Normalize(
    InFieldName = embr_rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Combined&nbsp;Ember&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

dens_dev = WeightedSum(
    InFieldNames = [nm_dns_1kf, nm_dns_0p5, nm_dns_3mi], 
    Weights = [0.33, 0.33, 0.34],
    Metadata = [
        DisplayName: Proximity&nbsp;To&nbsp;Dense&nbsp;Development&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

nm_den_dev = Normalize(
    InFieldName = dens_dev, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName:  Proximity&nbsp;To&nbsp;Dense&nbsp;Development,
        ColorMap: RdYlBu_r
    ]
)

Hi_Fr_Rsk = WeightedSum(
    InFieldNames = [Nm_Lg_Fire, Nm_Fire_Ig], 
    Weights = [0.5, 0.5],
    Metadata = [
        DisplayName: High&nbsp;Fire&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fr_Rsk = Normalize(
    InFieldName = Hi_Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Normalized&nbsp;High&nbsp;Fire&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

WUI_Rsk = Maximum(
    InFieldNames = [WUI_Interm, WUI_Interf],
    Metadata = [
        DisplayName: WUI&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

Hi_Pri = WeightedSum(
    InFieldNames = [Nm_Fr_Rsk, WUI_Rsk, LU_Cat_Cvt], 
    Weights = [0.333, 0.333, 0.334],
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Area,
        ColorMap: RdYlBu_r
    ]
)

Nm_Hi_Pri = Normalize(
    InFieldName = Hi_Pri, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Normalized&nbsp;High&nbsp;Priority&nbsp;Area,
        ColorMap: RdYlBu_r
    ]
)

# Output

Out = EEMSWrite(
  OutFileName = {output_path},
  OutFieldNames = [Join_ID, Nm_Hi_Pri, Hi_Pri, Nm_Fr_Rsk, Hi_Fr_Rsk, WUI_Rsk, Nm_Lg_Fire, Nm_Fire_Ig, WUI_Interm, WUI_Interf, Pr_Lg_Fr, Pr_Fire_Ig, LU_Cat_Cvt, LU_Cat_Fz, Mj_LU_Cat]
)
    """
    return base_str

In [32]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/A_RU_9.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output.csv")
base_str = get_base_str(reporting_units_path, output_path)
f = open("EEMS_CMD_FILE.mpt", "w")
f.write(base_str)
f.close()

**Run EEMS Command `mpilot eems-csv [command_file]`**

**Now we need to merge the outputted EEMS CSV with the rest of the data and create a shapefile for it. Also note we need to project it to UTM Zone 11N so that it visualizes in EEMS Online**

In [33]:
dir_path = os.path.join(THIS_FOLDER, "out_data/final")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

eems = pd.merge(pd.read_csv("out_data/eems/EEMS_Output.csv"), pd.read_csv("out_data/eems/A_RU_5.csv")[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
crs_proj = CRS("EPSG:2229")
gdf = gpd.GeoDataFrame(eems, crs=crs_proj, geometry = eems.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/Final_EEMS_Output.shp")

**Now prep the Command File and the geodatabase for EEMS Online**

In [34]:
arcpy.ResetEnvironments()
gdb_name = "EEMS.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path
path = os.path.join(THIS_FOLDER, "out_data/final/Final_EEMS_Output.shp")
FeatureClass = arcpy.CopyFeatures_management(path, "EEMS")
crs = arcpy.SpatialReference('NAD 1983 UTM Zone 11N')
FeatureClass = arcpy.Project_management("EEMS", "EEMS_rpj", crs)
arcpy.management.Delete("EEMS")

<Result 'true'>

In [35]:
f1 = open("EEMS_CMD_FILE.mpt", 'r')
f2 = open("EEMS_CMD_FILE_2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

In [36]:
arcpy.ClearWorkspaceCache_management()
del FeatureClass

**Now delete the EEMSWRITE command from the Command File and zip the geodatabase with the final EEMS feature class (and only that feature class)**

In [37]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 18180 entries, 0 to 18179
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Join_ID     18180 non-null  float64 
 1   Nm_Hi_Pri   18180 non-null  float64 
 2   Hi_Pri      18180 non-null  float64 
 3   Nm_Fr_Rsk   18180 non-null  float64 
 4   Hi_Fr_Rsk   18180 non-null  float64 
 5   WUI_Rsk     18180 non-null  float64 
 6   Nm_Lg_Fire  18180 non-null  float64 
 7   Nm_Fire_Ig  18180 non-null  float64 
 8   WUI_Interm  18180 non-null  float64 
 9   WUI_Interf  18180 non-null  float64 
 10  Pr_Lg_Fr    18180 non-null  float64 
 11  Pr_Fire_Ig  18180 non-null  float64 
 12  LU_Cat_Cvt  18180 non-null  float64 
 13  LU_Cat_Fz   18180 non-null  float64 
 14  Mj_LU_Cat   18180 non-null  float64 
 15  geometry    18180 non-null  geometry
dtypes: float64(15), geometry(1)
memory usage: 2.4 MB


In [39]:
gdf.tail()

,Join_ID,Nm_Hi_Pri,Hi_Pri,Nm_Fr_Rsk,Hi_Fr_Rsk,WUI_Rsk,Nm_Lg_Fire,Nm_Fire_Ig,WUI_Interm,WUI_Interf,Pr_Lg_Fr,Pr_Fire_Ig,LU_Cat_Cvt,LU_Cat_Fz,Mj_LU_Cat,geometry
18175,18176.0,0.147312,0.146023,0.438507,0.395918,0.0,0.312374,0.479462,0.0,0.0,22.431718,40.337782,0.0,0.0,1.0,"POLYGON Z ((5960742.195 2055224.228 0.000, 596..."
18176,18177.0,0.166440,0.164983,0.495445,0.447326,0.0,0.233804,0.660847,0.0,0.0,16.789604,55.597952,0.0,0.0,1.0,"POLYGON Z ((6066369.965 2169105.171 0.000, 606..."
18177,18178.0,0.168017,0.166546,0.500138,0.451564,0.0,0.224598,0.678530,0.0,0.0,16.128485,57.085602,0.0,0.0,1.0,"POLYGON Z ((6121653.764 2126234.839 0.000, 612..."
18178,18179.0,0.190515,0.188848,0.466809,0.421472,0.0,0.265911,0.577033,0.0,0.0,19.095182,48.546550,0.1,0.1,2.0,"POLYGON Z ((6128059.221 2123165.174 0.000, 612..."
18179,18180.0,0.165504,0.164056,0.492660,0.444811,0.0,0.231212,0.658410,0.0,0.0,16.603491,55.392937,0.0,0.0,1.0,"POLYGON Z ((6118657.825 2132147.687 0.000, 611..."
